## Modeling “[The Probability of 4d6, Drop the Lowest, Reroll 1s](http://prestonpoulter.com/2010/11/19/the-probability-of-4d6-drop-the-lowest-reroll-1s/)” in [``dyce``](https://posita.github.io/dyce/)

Select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.1"]
        try:
            import piplite ; await piplite.install(requirements)
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H, P
from itertools import chain

p_4d6 = 4 @ P(6)
d6_reroll_first_one = H(6).substitute(
    lambda h, outcome: h if outcome == 1 else outcome
)
p_4d6_reroll_first_one = 4 @ P(d6_reroll_first_one)
p_4d6_reroll_all_ones = 4 @ P(H(5) + 1)

markers = "Ds^*xo"
attr_results: dict[str, H] = {
    "3d6": 3 @ H(6),  # marker="D"
    "4d6 - discard lowest": p_4d6.h(slice(1, None)),  # marker="s"
    "4d6 - re-roll first 1, discard lowest": p_4d6_reroll_first_one.h(
        slice(1, None)
    ),  # marker="^"
    "4d6 - re-roll all 1s (i.e., 4d5 + 1), discard lowest": p_4d6_reroll_all_ones.h(
        slice(1, None)
    ),  # marker="*"
    "2d6 + 6": 2 @ H(6) + 6,  # marker="x"
    "4d4 + 2": 4 @ H(4) + 2,  # marker="o"
}
zero_fill_outcomes = set(chain(*(res.outcomes() for res in attr_results.values())))
attr_results = {
    label: res.zero_fill(zero_fill_outcomes) for label, res in attr_results.items()
}

In [3]:
from anydyce.viz import BreakoutType, jupyter_visualize

jupyter_visualize(
    [(label, res) for label, res in attr_results.items()],
    default_breakout_type=BreakoutType.BURST,
    default_markers=markers,
)